<a href="https://colab.research.google.com/github/manognadeva/StoryWeaver/blob/main/NLP_Final_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyvis
!pip install rapidfuzz
!pip install gensim
!pip install pyLDAvis

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 16.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 54.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.6/2.6 MB 38.2 MB/s eta 0:00:00


In [4]:
import requests
import csv
from io import StringIO
from pathlib import Path
import pandas as pd
import re
import random
import numpy as np
import tqdm
from rapidfuzz import fuzz, process

import spacy
from pyvis.network import Network
from IPython.core.display import display, HTML
import networkx as nx

from collections import defaultdict

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

# Gensim
import gensim
import gensim.corpora as corpora
from gensim import utils
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim

nltk.download('stopwords')
nltk.download('punkt_tab')

import warnings
warnings.filterwarnings('ignore')

try:
    from dateparser.search import search_dates
except ImportError:
    !pip install dateparser
    from dateparser.search import search_dates

import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.7/295.7 kB 10.8 MB/s eta 0:00:00


In [5]:
def dicts_to_html_table(dicts):
    html = []
    header = None
    for d in dicts:
        if header is None:
            header = d.keys()
            html.append("<table><tr>" +
                        "".join([f"<th>{h}</th>" for h in header]) +
                        "</tr>")
        html.append("<tr>" +
                    "".join([f"<td>{d[h]}</td>" for h in header]) +
                    "</tr>")
    html.append("</table>")

    return "".join(html)

In [6]:
def get_fiction_books_list():
  GUTENBERG_CSV_URL = "https://www.gutenberg.org/cache/epub/feeds/pg_catalog.csv.gz"

  r = requests.get(GUTENBERG_CSV_URL)
  csv_text = r.content.decode("utf-8")

  print(f"Total size (Catalog): {len(r.content) / 1024**2:0.2f}MB")

  fiction_books = [book for book in csv.DictReader(StringIO(csv_text))
                   if 'fiction' in book['Subjects'].lower()]

  fiction_books_english = [b for b in fiction_books
                   if 'fiction' in b['Subjects'].lower()
                   and "Indexes" not in b["Subjects"]
                   and b["Type"] == "Text"
                   and 'en' in b['Language']]
  return fiction_books_english

In [7]:
def strip_headers(text):
    in_text = False
    output = []

    for line in text.splitlines():
        if GUTENBERG_TEXT in line:
            if not in_text:
                in_text = True
            else:
                break
        else:
            if in_text:
                output.append(line)

    return "\n".join(output).strip()

In [8]:
def book_text(book_id):
    r = requests.get(GUTENBERG_TEXT_URL.format(id=book_id))
    text = r.text
    clean_text = strip_headers(text)
    return clean_text

In [9]:
def download_books(fiction_books_english,data_path,num_of_books):
  i=0
  for book in fiction_books_english:
      if i <=num_of_books:
        i+=1
        id = book["Text#"]
        text = book_text(id)
        print(f"Saving {book['Title']} by {book['Authors']} containing {len(text):_} characters")
        with open(data_path / (str(id) + ".txt"), "wt") as f:
            f.write(text)
  metadata_file_path = data_path/'metadata.csv'
  with open(metadata_file_path, 'wt') as f:
    csv_writer = csv.DictWriter(f, fieldnames=fiction_books_english[0].keys())
    csv_writer.writeheader()
    i=0
    for book in fiction_books_english:
      if i<=num_of_books:
        i+=1
        csv_writer.writerow(book)

  return metadata_file_path

In [10]:
def preprocess_text(text, processed_folder_path,book_id):
  contents_index = re.search(r"contents", text.lower(), re.IGNORECASE)
  matches = list(re.finditer(r"chapter", text.lower()))
  print(len(matches))

  processed_folder_path_book = processed_folder_path / Path(str(book_id))
  # processed_folder_path_book = processed_folder_path / Path("alice-in-wonderland")
  processed_folder_path_book.mkdir(exist_ok=True)
  if len(matches)>0:
    book_extract = text[matches[(len(matches)//2)].start():]
    Page_1 = text[:matches[(len(matches)//2)].start()]
    with open(processed_folder_path_book/ ("book_excerpt.txt"), 'w') as f:
      f.write(book_extract)
    with open(processed_folder_path_book/ ("index_page.txt"), 'w') as f:
      f.write(Page_1)
  else:
    with open(processed_folder_path_book/ ("book_excerpt.txt"), 'w') as f:
      f.write(text)
    with open(processed_folder_path_book/ ("index_page.txt"), 'w') as f:
      f.write(text)

In [11]:
def process_book_into_index_and_excerpt(books_folder_path,processed_folder_path, book_id):
  #Split the text and create a list
  with open(books_folder_path / f"{book_id}.txt", 'r') as f:
    data = f.read()

  data = data.replace('\n',' ')
  data = data.replace('\r',' ')

  preprocess_text(data, processed_folder_path, book_id)

In [12]:
def get_chapters(processed_folder_path, book_id):
  with open(processed_folder_path / f"{book_id}/index_page.txt", 'r') as f:
    index = f.read()
  try:
    chapter_pattern = r'(?i)(CHAPTER [IVXLCDM]+[.:]?)'
    chapters_num = re.findall(chapter_pattern, index)
    titles = re.split(r'(CHAPTER [IVXLCDM]+[.:]?)', index)[1:]
    if titles==[]:
      titles = re.split(r'(Chapter [IVXLCDM]+[.:]?)', index)[1:]
    titles = [titles[i + 1] for i in range(0, len(titles), 2)]
    titles = [chapter.strip() for chapter in titles]


    df = pd.DataFrame({"Chapter No.":chapters_num,'Title':titles})
    df.to_csv(processed_folder_path/ f"{book_id}/Chapters_list.csv",index=False)
  except Exception as e:
    chapters_num =[]
    titles = []
    df = pd.DataFrame({"Chapter No.":chapters_num,'Title':titles})
    df.to_csv(processed_folder_path/ f"{book_id}/Chapters_list.csv",index=False)


In [13]:
def remove_stop_words(input_string):

  stop_words = set(stopwords.words('english'))
  words = word_tokenize(input_string)
  filtered_words = [word for word in words if word.lower() not in stop_words]
  filtered_string = ' '.join(filtered_words)

  return filtered_string

In [14]:
def remove_characters(data):
  data = data.replace('\'',' ')
  data = data.replace('_',' ')
  data = data.replace('-',' ')
  data = data.replace(';',' ')
  data = data.replace(':',' ')
  data = data.replace('\"',' ')
  data = data.replace(',',' ')
  data = data.replace('?',' ')
  data = data.replace('!',' ')
  data = data.replace('.',' ')
  data = data.replace('(',' ')
  data = data.replace(')',' ')
  data = data.replace('“',' ')
  data = data.replace('”',' ')
  data = data.replace('‘',' ')
  data = data.replace('’',' ')
  data = data.replace('—',' ')
  return data

In [15]:
def remove_chapter_header_from_book_excerpt(processed_folder_path,book_id):
  with open(processed_folder_path / f"{book_id}/book_excerpt.txt", 'r') as f:
    data = f.read()
  chapter = pd.read_csv(processed_folder_path/f"{book_id}/Chapters_list.csv")

  # chapter['Combined'] = chapter['Chapter No.'] + ' ' + chapter['Title']
  cleaned_text = data
  if len(chapter) > 0:
    for _, row in chapter.iterrows():
        # cleaned_text = re.sub(re.escape(str(row['Combined'])), '', cleaned_text, flags=re.IGNORECASE)
        cleaned_text = re.sub(re.escape(str(row['Chapter No.'])), '', cleaned_text, flags=re.IGNORECASE)
        cleaned_text = re.sub(re.escape(str(row['Title'])), '', cleaned_text, flags=re.IGNORECASE)
    # cleaned_text = cleaned_text.lower()
    cleaned_text = remove_stop_words(cleaned_text)
    cleaned_text = remove_characters(cleaned_text)
  return cleaned_text

In [16]:
def get_sample(sample_size, parsed_book):
  my_sample = random.sample(list(parsed_book.sents),sample_size) # select a random sample of sentences
  sample=[]
  for sent in my_sample:
      sent = re.sub("\s+"," ",sent.text) # clean up the whitespace
      sample.append(sent)
  return sample

In [17]:
def named_entity_recognition(sample):
  entities=[]
  type_entity=[]
  sentences=[]
  for sent in sample:
      parsed_sentence=nlp(sent)
      for ent in parsed_sentence.ents:
          if ent.text not in entities:
              entities.append(ent.text)
              sentences.append(sent)
              type_entity.append(ent.label_)
  Entities=pd.DataFrame({'Sentence':sentences,'Entity':entities,'Entity_type':type_entity})
  print('The total number of entities detected were:{}'.format(len(Entities)))
  return Entities

In [18]:
def recognition(data):
  nlp = spacy.load('en_core_web_sm')
  parsed_book = nlp(data)
  sample = get_sample(len(list(parsed_book.sents)),parsed_book)
  entities_detected = named_entity_recognition(sample)
  return entities_detected

In [19]:
def get_characters(entities_detected, processed_path, book_id):
  characters_df = entities_detected[entities_detected['Entity_type'].isin(["PERSON","ORG"])]
  characters_df['Entity'] = characters_df['Entity'].apply(lambda x: re.sub("[\(].*[\)]","",x))
  characters_df['Character_firstname'] = characters_df['Entity'].apply(lambda x: x.split(" ")[0])
  characters_df.drop_duplicates(subset='Character_firstname',inplace=True)
  characters_df=characters_df[['Entity','Character_firstname']]
  remove_words = ['which','would','she','he','fit','down', 'adventures','begin','stuff','latitude','longitude','through','ful','the']
  characters_df = characters_df[~characters_df['Character_firstname'].str.lower().isin([word.lower() for word in remove_words])]
  characters_df.to_csv(processed_path / f"{book_id}/characters.csv",index=False)
  return characters_df

In [20]:
def filter_entity(ent_list,character_df):
  return [ent for ent in ent_list
          if ent in character_df['Entity'].values
          or ent in character_df['Character_firstname'].values]

In [21]:
def get_character_relationships(parsed_book, character_df):
  sent_entity_df = []

  for sent in parsed_book.sents:
    entity_list = [ent.text for ent in sent.ents]
    sent_entity_df.append({"sentence":sent,"entities":entity_list})
  sent_entity_df = pd.DataFrame(sent_entity_df)
  sent_entity_df['filtered_entities'] = sent_entity_df['entities'].apply(lambda x: filter_entity(x,character_df))

  sent_entity_df_filtered = sent_entity_df[sent_entity_df['filtered_entities'].apply(lambda x: len(x) > 0)]

  window_size = 5
  relationships = []

  for i in range(sent_entity_df_filtered.index[-1]):
    end_i = min(i+window_size,sent_entity_df_filtered.index[-1])
    char_list = sum((sent_entity_df_filtered.loc[i:end_i].filtered_entities),[])

    char_unique = [char_list[i] for i in range(len(char_list))
                  if (i==0) or (char_list[i] != char_list[i-1])]

    if len(char_unique) > 2:
      for idx, a in enumerate(char_unique[:-1]):
        for b in char_unique[idx+1:]:
          relationships.append({"source":a,"target":b})
  relationships_df = pd.DataFrame(relationships)
  relationships_df = pd.DataFrame(np.sort(relationships_df.values,axis=1),columns = relationships_df.columns)
  relationships_df = relationships_df[relationships_df["source"] != relationships_df["target"]]
  relationships_df['value'] = 1
  relationships_df = relationships_df.groupby(['source','target']).sum().reset_index()
  relationships_df = relationships_df.sort_values(by='value', ascending=False)

  if len(relationships_df) > 500:
    relationships_df = relationships_df.head(500)

  return relationships_df

In [22]:
def see_character_relationships(processed_folder_path,book_id,relationships_df):
  G = nx.from_pandas_edgelist(relationships_df,source='source',target='target',edge_attr='value', create_using=nx.Graph())
  net = Network(notebook=True, width="1000px", height="700px", bgcolor="#222222", font_color="white", cdn_resources='remote')
  node_degree = dict(G.degree)
  nx.set_node_attributes(G, node_degree, 'size')
  net.from_nx(G)
  path = str(processed_folder_path) + f'/{book_id}/network.html'
  net.show(path)
  display(HTML(path))


In [23]:
def get_locations(entities_detected, processed_path,book_id):
  locations = entities_detected[entities_detected['Entity_type'].isin(["GPE", "LOC"])]
  locations['Location'] = locations['Entity'].apply(lambda x: x.split(" ")[0])
  locations.drop_duplicates(subset='Entity',inplace=True)
  locations = locations[["Location"]]
  locations.to_csv(processed_path / f"{book_id}/locations.csv",index=False)
  return locations

In [24]:
def timeline_creation(data_path,book_id,book_text: str):
    settings = {
    'PREFER_DATES_FROM': 'past',
    'STRICT_PARSING': True,
    'RETURN_AS_TIMEZONE_AWARE': False
    }
    nlp = spacy.load("en_core_web_sm")
    doc = nlp(book_text)
    sentences = [sent.text.strip() for sent in doc.sents]
    anchor_year = None
    timeline = []
    events_list = []
    problem_sentences = []
    for sentence in sentences:
        sentence = re.sub(r'[^\w\s]', '', sentence)
        try:
          dates = search_dates(sentence, settings=settings)
          if dates:
              for date_text, date in dates:
                  if date.year and "year" in sentence.lower():
                      anchor_year = date.year
                  if not date.year and anchor_year:
                      date = date.replace(year=anchor_year)
                  if date.year == 2024 and "2024" not in sentence:
                      continue
                  if sentence not in events_list:
                    events_list.append(sentence)
                    timeline.append({"Event": sentence, "Timeline": date.strftime('%B %Y') if date.year else date.strftime('%B')})
        except RecursionError:
          problem_sentences.append(sentence)


    timeline_df = pd.DataFrame(timeline)


    print("\nTimeline of the Story:")

    timeline_df = timeline_df.drop_duplicates()
    timeline_df.to_csv(data_path/ f"{book_id}/timeline.csv",index=False)
    print(timeline_df)

    return timeline_df

In [25]:
def sent_to_words(sentences):
  yield(gensim.utils.simple_preprocess(str(sentences), deacc=True))

def make_bigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    bigram = gensim.models.Phrases(texts, min_count=5, threshold=100)
    bigram_mod = gensim.models.phrases.Phraser(bigram)
    trigram = gensim.models.Phrases(bigram[texts], threshold=100)
    trigram_mod = gensim.models.phrases.Phraser(trigram)
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [26]:
def calculate_preplexity_and_coherence(corpus, lda_model,data_lemmatized, id2word):
  # Compute Perplexity
  Perplexity = lda_model.log_perplexity(corpus)
  print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

  # Compute Coherence Score
  coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
  coherence_lda = coherence_model_lda.get_coherence()
  print('\nCoherence Score: ', coherence_lda)
  return Perplexity, coherence_lda

In [27]:
def compute_coherence_values(corpus,data_lemmatized, id2word, k, a, b):

    lda_model = gensim.models.LdaMulticore(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=k,
                                           random_state=100,
                                           chunksize=100,
                                           passes=10,
                                           alpha=a,
                                           eta=b)

    coherence_model_lda = CoherenceModel(model=lda_model, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

    return coherence_model_lda.get_coherence()

In [28]:
def get_best_params(corpus, id2word, data_lemmatized, data_path, book_id):
  grid = {}
  grid['Validation_Set'] = {}

  # Topics range
  min_topics = 2
  max_topics = 11
  step_size = 1
  topics_range = range(min_topics, max_topics, step_size)

  # Alpha parameter
  alpha = list(np.arange(0.01, 1, 0.3))
  alpha.append('symmetric')
  alpha.append('asymmetric')

  # Beta parameter
  beta = list(np.arange(0.01, 1, 0.3))
  beta.append('symmetric')

  # Validation sets
  num_of_docs = len(corpus)
  corpus_sets = [gensim.utils.ClippedCorpus(corpus, int(num_of_docs*0.75)),
                corpus]

  corpus_title = ['75% Corpus', '100% Corpus']

  model_results = {'Validation_Set': [],
                  'Topics': [],
                  'Alpha': [],
                  'Beta': [],
                  'Coherence': []
                  }

  # Can take a long time to run
  if 1 == 1:
      pbar = tqdm.tqdm(total=(len(beta)*len(alpha)*len(topics_range)*len(corpus_title)))

      # iterate through validation corpuses
      for i in range(len(corpus_sets)):
          size = len(list(corpus_sets[i]))
          # iterate through number of topics
          for k in topics_range:
              # iterate through alpha values
              for a in alpha:
                  # iterare through beta values
                  for b in beta:
                      # get the coherence score for the given parameters
                      if size > 0:
                        cv = compute_coherence_values(corpus=corpus_sets[i],data_lemmatized=data_lemmatized, id2word=id2word,
                                                      k=k,a=a,b=b)
                        # Save the model results
                        model_results['Validation_Set'].append(corpus_title[i])
                        model_results['Topics'].append(k)
                        model_results['Alpha'].append(a)
                        model_results['Beta'].append(b)
                        model_results['Coherence'].append(cv)
                        # model_results['Size'].append(size)

                      pbar.update(1)
      pd.DataFrame(model_results).to_csv(data_path/ f"{book_id}/lda_tuning_results.csv",index=False)
      pbar.close()
  result = pd.read_csv(data_path/ f"{book_id}/lda_tuning_results.csv")
  best_row = result.loc[result['Coherence'].idxmax()]
  return best_row, corpus_sets, result

In [29]:
def build_best_lda_model(best_row, corpus_sets,data_lemmatized, id2word):
  alpha = float(best_row['Alpha'])
  beta = float(best_row['Beta'])
  topics = int(best_row['Topics'])
  if best_row['Validation_Set'] == '75% Corpus':
    corpus_set = corpus_sets[0]
  else:
    corpus_set = corpus_sets[1]
  lda_model = gensim.models.LdaMulticore(corpus=corpus_set,
                                            id2word=id2word,
                                            num_topics=topics,
                                            random_state=100,
                                            chunksize=100,
                                            passes=10,
                                            alpha=alpha,
                                            eta=beta)
  calculate_preplexity_and_coherence(corpus_set, lda_model,data_lemmatized, id2word)
  return lda_model, corpus_set

In [30]:
def lda_topics(data_words, data_path, book_id):
    data = list(sent_to_words(data_words))

    # Form Bigrams
    data_words_bigrams = make_bigrams(data)

    # Initialize spacy 'en' model, keeping only tagger component (for efficiency)
    # python3 -m spacy download en

    # Do lemmatization keeping only noun, adj, vb, adv
    data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

    # Create Dictionary
    id2word = corpora.Dictionary(data_lemmatized)

    # Create Corpus
    texts = data_lemmatized

    # Term Document Frequency
    corpus = [id2word.doc2bow(text) for text in texts]

    best_row, corpus_sets, result = get_best_params(corpus, id2word, data_lemmatized, data_path, book_id)
    lda_model, corpus_set = build_best_lda_model(best_row, corpus_sets,data_lemmatized, id2word)

    return lda_model, corpus_set, id2word


In [31]:
def show_vis(lda_model, corpus, dictionary, data_path, book_id):
  pyLDAvis.enable_notebook()
  vis = pyLDAvis.gensim.prepare(lda_model, corpus, dictionary, mds="mmds", R=30)
  path = str(data_path) + f'/{book_id}/lda_visualization.html'
  pyLDAvis.save_html(vis, path)
  display(HTML(path))

In [32]:
def topic_modeling(data_path,book_id, text, characters, num_topics=5, num_words=10):
  # Predefined genre keywords
  GENRE_KEYWORDS = {
      "Fiction": ["story", "novel", "imagination", "narrative", "character"],
      "Thriller": ["murder", "crime", "mystery", "suspense", "detective"],
      "Romance": ["love", "relationship", "passion", "romantic", "affection"],
      "Fantasy": ["magic", "wizard", "dragon", "kingdom", "myth"],
      "Adventure": ["journey", "explore", "quest", "travel", "danger"]
  }
  nlp = spacy.load('en_core_web_sm', disable=['parser', 'ner'])
  try:
    # Preprocessing: Tokenization and stopword removal

    filtered_tokens = word_tokenize(text.lower())
    filtered_tokens = [word for word in filtered_tokens if word.isalnum() and word not in characters]

    # Genre identification
    genre_counts = {genre: sum(filtered_tokens.count(keyword) for keyword in keywords)
                    for genre, keywords in GENRE_KEYWORDS.items()}
    identified_genre = max(genre_counts, key=genre_counts.get)

    # Topic modeling
    lda_model, corpus, dictionary = lda_topics(text,data_path, book_id)


    topics = lda_model.print_topics(num_topics=num_topics, num_words=num_words)
    formatted_topics = []
    for topic_id, words in topics:
        topic = f"Topic {topic_id + 1}: " + ", ".join(
            [word.split('*')[1].strip().strip('"') for word in words.split(' + ') if word.split('*')[1].strip().strip('"') not in characters]
        )
        formatted_topics.append(topic)

    show_vis(lda_model, corpus, dictionary, data_path, book_id)
    Topics = {"Genre": identified_genre,
            "Topics": "; ".join(formatted_topics)}

    Topics_df = pd.DataFrame(Topics, index=[0])
    Topics_df.to_csv(data_path/ f"{book_id}/Topics.csv",index=False)
    return identified_genre, formatted_topics

  except Exception as e:
    print(f"Error Doing Topic modelling: {e}")
    Topics = {"Genre": "Unknown Genre",
            "Topics": "Error in processing Topics"}

    Topics_df = pd.DataFrame(Topics, index=[0])
    Topics_df.to_csv(data_path / f"{book_id}/Topics.csv",index=False)

    return "Unknown Genre", "Error in processing Topics"

In [33]:
GUTENBERG_TEXT_URL = "https://www.gutenberg.org/ebooks/{id}.txt.utf-8"
GUTENBERG_TEXT = "PROJECT GUTENBERG EBOOK "
num_of_books = 10

In [34]:
data_path = Path("books")
data_path.mkdir(exist_ok=True)
processed_folder_path = Path("/content/processed_books/")
processed_folder_path.mkdir(exist_ok=True)

In [35]:
fic_books = get_fiction_books_list()

Total size (Catalog): 19.43MB


In [36]:
metadata_file = download_books(fic_books,data_path,num_of_books)

Saving Alice's Adventures in Wonderland by Carroll, Lewis, 1832-1898 containing 144_599 characters
Saving Through the Looking-Glass by Carroll, Lewis, 1832-1898 containing 164_249 characters
Saving Moby-Dick; or, The Whale by Melville, Herman, 1819-1891 containing 1_218_709 characters
Saving Peter Pan by Barrie, J. M. (James Matthew), 1860-1937 containing 255_583 characters
Saving O Pioneers! by Cather, Willa, 1873-1947 containing 299_678 characters
Saving Far from the Madding Crowd by Hardy, Thomas, 1840-1928 containing 773_819 characters
Saving Herland by Gilman, Charlotte Perkins, 1860-1935 containing 296_895 characters
Saving The Scarlet Letter by Hawthorne, Nathaniel, 1804-1864 containing 480_298 characters
Saving The Time Machine by Wells, H. G. (Herbert George), 1866-1946 containing 179_692 characters
Saving The War of the Worlds by Wells, H. G. (Herbert George), 1866-1946 containing 337_430 characters
Saving The Legend of Sleepy Hollow by Irving, Washington, 1783-1859 containin

In [37]:
books = pd.read_csv(metadata_file)

In [ ]:
books

,Text#,Type,Issued,Title,Language,Authors,Subjects,LoCC,Bookshelves
0,11,Text,2008-06-27,Alice's Adventures in Wonderland,en,"Carroll, Lewis, 1832-1898",Fantasy fiction; Children's stories; Imaginary...,PR; PZ,Children's Literature; Browsing: Children & Yo...
1,12,Text,2008-06-25,Through the Looking-Glass,en,"Carroll, Lewis, 1832-1898",Fantasy fiction; Children's stories; Imaginary...,PR; PZ,Children's Literature; Best Books Ever Listing...
2,15,Text,1991-05-01,"Moby-Dick; or, The Whale",en,"Melville, Herman, 1819-1891",Whaling -- Fiction; Sea stories; Psychological...,PS,Adventure; Best Books Ever Listings; Browsing:...
3,16,Text,2008-06-25,Peter Pan,en,"Barrie, J. M. (James Matthew), 1860-1937",Fantasy literature; Peter Pan (Fictitious char...,PR; PZ,Children's Literature; Movie Books; Browsing: ...
4,24,Text,2008-06-27,O Pioneers!,en,"Cather, Willa, 1873-1947",Nebraska -- Fiction; Historical fiction; Front...,PS,Browsing: Culture/Civilization/Society; Browsi...
5,27,Text,1992-03-01,Far from the Madding Crowd,en,"Hardy, Thomas, 1840-1928",Didactic fiction; Love stories; Triangles (Int...,PR,Browsing: Culture/Civilization/Society; Browsi...
6,32,Text,2008-06-27,Herland,en,"Gilman, Charlotte Perkins, 1860-1935",Utopias -- Fiction; Women -- Fiction; Utopian ...,PS,Best Books Ever Listings; Browsing: Culture/Ci...
7,33,Text,1992-06-01,The Scarlet Letter,en,"Hawthorne, Nathaniel, 1804-1864",Adultery -- Fiction; Historical fiction; Reven...,PS,Harvard Classics; Movie Books; Browsing: Cultu...
8,35,Text,2004-10-02,The Time Machine,en,"Wells, H. G. (Herbert George), 1866-1946",Science fiction; Time travel -- Fiction; Dysto...,PR,Science Fiction; Movie Books; Browsing: Litera...
9,36,Text,2004-10-01,The War of the Worlds,en,"Wells, H. G. (Herbert George), 1866-1946",Science fiction; War stories; Martians -- Fict...,PR,Science Fiction; Movie Books; Browsing: Histor...


In [38]:
def process_book(data_path, processed_folder_path, book_id):
  process_book_into_index_and_excerpt(data_path,processed_folder_path, book_id)
  get_chapters(processed_folder_path, book_id)
  data = remove_chapter_header_from_book_excerpt(processed_folder_path, book_id)
  parsed_book = nlp(data)
  entities_detected = recognition(data)
  locations = get_locations(entities_detected,processed_folder_path, book_id)
  characters = get_characters(entities_detected,processed_folder_path,book_id)
  char_relations = get_character_relationships(parsed_book,characters)
  see_character_relationships(processed_folder_path, book_id, char_relations)
  df_timeline = timeline_creation(processed_folder_path, book_id, data)
  genre, topics = topic_modeling(processed_folder_path,book_id, data, list(characters["Character_firstname"].str.lower()))

In [39]:
book_id = books['Text#'][5]

In [ ]:
nlp = spacy.load('en_core_web_sm')
process_book(data_path, processed_folder_path, book_id)

118
The total number of entities detected were:1413
/content/processed_books/27/network.html
